In [1]:
import pandas as pd
import numpy as np
import git
from datetime import datetime as dt


In [2]:
repo = git.Repo(".", search_parent_directories=True).git.rev_parse("--show-toplevel")
orig_rides_df = pd.read_csv(f"{repo}/data/rides_combined.csv")
orig_rides_df.columns

c:\Users\jostm7\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (8,37,45,46,47) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Index(['Unnamed: 0', 'id', 'user_id', 'distance', 'number_of_passenger',
       'price_operations', 'price_offer', 'price_payed', 'free_ride',
       'payment_type', 'pickup_address', 'dropoff_address', 'state',
       'created_from_offer', 'created_at', 'scheduled_to', 'dispatched_at',
       'pickup_arrival_time', 'arriving_push', 'vehicle_arrived_at',
       'earliest_pickup_expectation', 'pickup_first_eta', 'pickup_eta',
       'pickup_at', 'dropoff_first_eta', 'dropoff_eta', 'dropoff_at',
       'updated_at', 'arrival_deviation', 'waiting_time', 'boarding_time',
       'ride_time', 'trip_time', 'shortest_ridetime', 'delay',
       'longer_route_factor', 'arrival_indicator', 'rating',
       'rating_puenktlichkeit', 'rating_sauberkeit', 'rating_fahrer',
       'rating_find_modstop', 'rating_other_comments', 'cancellation_reason',
       'cancellation_comment', 'bahn_card_number', 'year_card_type',
       'year_card_number', 'canceled_at', 'rating_question_one',
       'rating_quest

In [3]:
orig_external_df = pd.read_excel(f"{repo}/data/vehicle_data/Autofleet_Rides with External ID_2021+2022-05-15.xlsx")
orig_raw_df = pd.read_excel(f"{repo}/data/vehicle_data/MoD_Raw Data_2021+2022-05-15.xlsx")
orig_vehicle_usage_df = pd.read_excel(f"{repo}/data/vehicle_data/MoD_Vehicle Usage_2021+2022-05-15.xlsx")


In [4]:
print(f'Shape orig_rides_df: {orig_rides_df.shape}')
print(f'Shape orig_external_df: {orig_external_df.shape}')
print(f'Shape orig_raw_df: {orig_raw_df.shape}')
print(f'Shape orig_vehicle_usage_df: {orig_vehicle_usage_df.shape}')

Shape orig_rides_df: (18980, 51)
Shape orig_external_df: (18148, 32)
Shape orig_raw_df: (36393, 36)
Shape orig_vehicle_usage_df: (173041, 12)


In [5]:
rides_df = orig_rides_df
external_df = orig_external_df
raw_df = orig_raw_df
vehicle_usage_df = orig_vehicle_usage_df

In [6]:
print(rides_df['id'].isin(external_df['External Id']).unique())
print(rides_df['id'].isin(external_df['Id']).unique())
print(rides_df['id'].isin(vehicle_usage_df['Ride Id']).unique()) # Not unique because of Nan Values
print(vehicle_usage_df['Ride Id'].isin(external_df['External Id']).unique())
print(vehicle_usage_df['Ride Id'].isin(external_df['Id']).unique()) # Not unique because of Nan Values
print(vehicle_usage_df['Vehicle Id'].isin(external_df).unique())

[ True False]
[False]
[False  True]
[False  True]
[False  True]
[False]


In [7]:
filt_rides = rides_df[rides_df['id'].isin(external_df['External Id'])]
filt_rides_2 = rides_df[rides_df['id'].isin(vehicle_usage_df['Ride Id'])]
filt_rides_3 = rides_df[rides_df['id'].isin(raw_df['Ride External Id'])]

filt_vehicle_usage = vehicle_usage_df[vehicle_usage_df['Ride Id'].isin(external_df['Id'])]
filt_vehicle_usage_2 = vehicle_usage_df[vehicle_usage_df['Ride Id'].isin(external_df['External Id'])]
filt_vehicle_usage_3 = vehicle_usage_df[vehicle_usage_df['Ride Id'].isin(raw_df['Ride Id'])]


print(f"Matches between combined rides and autofleet_external id: {filt_rides['id'].count()}")
print(f"Matches between combined rides and raw_id: {filt_rides_3['id'].count()}")
print(f"Match values between combined_rides and vehicle_usage: {filt_rides_2['id'].unique()}")
print(f"Matches between vehicle_usage and autofleet_id: {filt_vehicle_usage['Ride Id'].count()}")
print(f"Matches between vehicle_usage and raw_id: {filt_vehicle_usage_3['Ride Id'].count()}")
print(f"Match values between vehicle_usage_id and autofleet_external id: {filt_vehicle_usage_2['Ride Id'].unique()}")

Matches between combined rides and autofleet_external id: 9496
Matches between combined rides and raw_id: 9496
Match values between combined_rides and vehicle_usage: [nan]
Matches between vehicle_usage and autofleet_id: 103910
Matches between vehicle_usage and raw_id: 103910
Match values between vehicle_usage_id and autofleet_external id: [nan]


In [8]:
print(external_df.columns)
# print(raw_df.columns)
print(vehicle_usage_df.columns)

Index(['Id', 'External Id', 'Ride Type', 'Matching Type', 'Dispatch Type',
       'Schedule Time (UTC)', 'Status', 'Rejection reason',
       'Contact person/passenger name', 'Contact person/passenger phone',
       'Demand Source name', 'Driver name', 'Driver external ID',
       'Vehicle plate', 'Price', 'Currency', 'Pooling', 'Rating',
       'Arrived to Pickup (UTC)', 'Pickup Completed (UTC)', 'Pickup address',
       'Pickup coordinates', 'Arrived to Dropoff (UTC)',
       'Dropoff Completed (UTC)', 'Dropoff address', 'Dropoff coordinates',
       'Actual Duration of Ride (min)', 'Planned Distance (m)',
       'Number Of Passengers', 'Number Of Items', 'Created by',
       'Created at (UTC)'],
      dtype='object')
Index(['Vehicle Id', 'Ride Type', 'Ride Id', 'Stop Point Type',
       'Stop Point Id', 'Stop Point status', 'Stop point completed (UTC)',
       'Vehicle plate', 'Lat', 'Lng', 'Actual Distance of Ride (m)',
       'Odometer Reading (m)'],
      dtype='object')


In [30]:
# vehicle_usage_df preprocessing - filteirng on Stop Point type and status + drop remaining duplicates

merge_vehicle_df = vehicle_usage_df[(vehicle_usage_df["Stop Point Type"] == "dropoff") & (vehicle_usage_df["Stop Point status"] == "completed")]
merge_vehicle_df.dropna(subset=["Ride Id"], inplace= True)
merge_vehicle_df.sort_values(by="Vehicle Id", inplace = True)
merge_vehicle_df.drop_duplicates(subset=["Ride Id"], inplace= True)
# dupl_vehicle_df = merge_vehicle_df[merge_vehicle_df.duplicated( subset=["Ride Id"] , keep= False)]
print(f"shape merge_vehicle_df: {merge_vehicle_df.shape}")
merge_vehicle_df

shape merge_vehicle_df: (14909, 12)


c:\Users\jostm7\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,Vehicle Id,Ride Type,Ride Id,Stop Point Type,Stop Point Id,Stop Point status,Stop point completed (UTC),Vehicle plate,Lat,Lng,Actual Distance of Ride (m),Odometer Reading (m)
21221,54fc7c8b-940f-4ba6-abda-9237be36e57a,passenger,37e102b9-5f5d-4938-88fd-0e16884c63af,dropoff,57679c3c-5b7b-42b1-b1e3-2751412b941a,completed,2021-12-04 09:09:07,NW-MD-3E,4.934618e+15,81419394493722,2488.0,94246180.0
23503,54fc7c8b-940f-4ba6-abda-9237be36e57a,passenger,50681c5c-4cb8-4d04-b80b-ea06c152f735,dropoff,11a564ac-fcd4-469e-ad4f-cc782efd6df6,completed,2021-11-09 07:55:12,NW-MD-3E,4.933748e+08,81315586,5725.0,94246180.0
23501,54fc7c8b-940f-4ba6-abda-9237be36e57a,passenger,06f0080a-fbc5-4cc7-8120-e92c15e759a6,dropoff,e48c008e-8601-4931-a906-ede47af60787,completed,2021-11-09 08:12:23,NW-MD-3E,4.935402e+08,81351358,3721.0,94246180.0
23499,54fc7c8b-940f-4ba6-abda-9237be36e57a,passenger,94a3310b-7e62-4e38-809d-b0f6103e8f04,dropoff,f7bda699-f311-4279-8950-51554cf39079,completed,2021-11-09 08:43:52,NW-MD-3E,4.934243e+08,81548779,3392.0,94246180.0
23497,54fc7c8b-940f-4ba6-abda-9237be36e57a,passenger,6470270d-975c-4b6a-b098-adf7e0050521,dropoff,e3d8e88a-cb30-4c06-b308-277a305b644e,completed,2021-11-09 10:31:05,NW-MD-3E,4.934129e+08,81611289,2824.0,94246180.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3415,fd0b8f54-e982-42eb-9d66-74d142807d9f,passenger,da4ed9a6-2737-4b5c-977b-8c275139bdb4,dropoff,1dbb476a-e8ec-4e35-97af-d34b148e1d9d,completed,2022-04-29 16:22:31,NW-MD-31E,4.932325e+16,812979355116079,4788.0,NaN
9326,fd0b8f54-e982-42eb-9d66-74d142807d9f,passenger,5dc43879-138f-4180-8fbb-6963a9d72a6d,dropoff,8c0a9882-37f9-4be5-b257-e111eb0b1966,completed,2022-03-26 13:10:13,NW-MD-31E,4.934257e+15,8140528153112360,1581.0,NaN
8048,fd0b8f54-e982-42eb-9d66-74d142807d9f,passenger,c0c95617-964d-4d72-8f48-a1c3a19c036b,dropoff,20c64010-f0bc-49c0-ae09-a6ab450b2c25,completed,2022-04-04 05:58:52,NW-MD-31E,4.935398e+15,8135189388449290,1295.0,NaN
12347,fd0b8f54-e982-42eb-9d66-74d142807d9f,passenger,2dc84931-774d-47ac-8b40-5f2acbf01e07,dropoff,c9c3d479-5028-4651-b1b5-ba683000da73,completed,2022-03-05 12:28:11,NW-MD-31E,4.929883e+15,820998249553163,6589.0,NaN


In [28]:
# external_df preprocessing
duplicated_external_df = external_df[external_df.duplicated( subset=["Id"] , keep= False)]
nan_external_df = external_df[external_df["Id"].isna()]
merge_external_df = external_df[external_df["Id"].isin(vehicle_usage_df['Ride Id'])]
merge_external_df
print(f"shape external_df: {external_df.shape}")
print(f"shape merge_external_df: {merge_external_df.shape}")



shape external_df: (18148, 32)
shape merge_external_df: (18104, 32)


In [29]:
vehicle_external_merge = vehicle_usage_df.merge(external_df,how='left',left_on="Ride Id",right_on="Id")
vehicle_external_merge

,Vehicle Id,Ride Type_x,Ride Id,Stop Point Type,Stop Point Id,Stop Point status,Stop point completed (UTC),Vehicle plate_x,Lat,Lng,...,Arrived to Dropoff (UTC),Dropoff Completed (UTC),Dropoff address,Dropoff coordinates,Actual Duration of Ride (min),Planned Distance (m),Number Of Passengers,Number Of Items,Created by,Created at (UTC)
0,57ab768a-59a3-45c5-a68f-8a5ef5139414,passenger,5b32c251-5897-4546-a237-e9f6c3bd1f74,pickup,997e2c6e-47d1-43a3-8218-948b5ef91885,completed,2022-05-15 11:24:43,NW-MD-10E,4.935157e+15,8156754450987930,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,57ab768a-59a3-45c5-a68f-8a5ef5139414,passenger,f61e1829-c99c-47a3-b2d2-10bb80423220,dropoff,5ab827cb-21ab-4564-8d5e-bf1b6eceb9ab,completed,2022-05-15 11:17:20,NW-MD-10E,4.936986e+16,8170304395148920,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,57ab768a-59a3-45c5-a68f-8a5ef5139414,passenger,f61e1829-c99c-47a3-b2d2-10bb80423220,pickup,6ccf5bb5-cebc-4cf9-997d-3e0a136a970a,completed,2022-05-15 11:11:14,NW-MD-10E,4.935038e+15,8164279999121560,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,57ab768a-59a3-45c5-a68f-8a5ef5139414,passenger,5b1080bb-299c-42f6-9cf7-7a4bc126e40e,dropoff,edd4a28a-f7ee-4be1-b534-fdf51e4e288a,completed,2022-05-15 11:03:29,NW-MD-10E,4.934010e+15,816794464468801,...,2022-05-15 11:03:16,2022-05-15 11:03:29,Le Quartier Hornbach,"[49.34, 8.168]",6.0,0,2.0,NaN,mod-service-account Service Account,2022-05-14 19:29:54
4,fd0b8f54-e982-42eb-9d66-74d142807d9f,passenger,72840ecc-ae05-4397-83d4-a42d16240c05,dropoff,fff03eb0-e005-4056-9fa9-89e053764cb6,completed,2022-05-15 11:00:34,NW-MD-31E,4.935046e+15,814095343575192,...,2022-05-15 10:59:15,2022-05-15 11:00:34,Hauptbahnhof,"[49.35, 8.141]",10.0,0,2.0,NaN,mod-service-account Service Account,2022-05-15 07:06:07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173036,NaN,passenger,4ba58357-ef2c-47fa-93b1-0441590332d8,pickup,e7506693-6c4f-4705-a30f-0f17e7eb6366,requested,NaT,NaN,NaN,NaN,...,2022-03-15 07:24:57,2022-03-15 07:24:58,Rathaus,"[49.354, 8.135]",7.0,3509,1.0,NaN,mod-service-account Service Account,2022-03-15 07:11:56
173037,NaN,passenger,27a42f6c-14cd-4dd5-8011-a0199b6067bc,pickup,0a6092c3-6627-4475-9ab2-0c7ca0a21d40,requested,NaT,NaN,NaN,NaN,...,2022-03-16 21:04:29,2022-03-16 21:05:09,Rathaus,"[49.354, 8.135]",6.0,3702,1.0,NaN,mod-service-account Service Account,2022-03-16 20:26:13
173038,NaN,passenger,f3b3901e-31cd-42f0-9e57-285dddad6e1a,dropoff,2e8a7fe9-2c93-4929-8bf9-94b88aa429fd,requested,NaT,NaN,NaN,NaN,...,2022-03-14 16:40:53,2022-03-14 16:40:54,Rathaus,"[49.354, 8.135]",13.0,5762,1.0,NaN,mod-service-account Service Account,2022-03-14 16:27:00
173039,NaN,passenger,4ba58357-ef2c-47fa-93b1-0441590332d8,dropoff,19e7b249-700a-4352-996e-9742bea13f9e,requested,NaT,NaN,NaN,NaN,...,2022-03-15 07:24:57,2022-03-15 07:24:58,Rathaus,"[49.354, 8.135]",7.0,3509,1.0,NaN,mod-service-account Service Account,2022-03-15 07:11:56


In [109]:
print(external_df.columns)
print(raw_df.columns)
print(vehicle_usage_df.columns)

Index(['Id', 'External Id', 'Ride Type', 'Matching Type', 'Dispatch Type',
       'Schedule Time (UTC)', 'Status', 'Rejection reason',
       'Contact person/passenger name', 'Contact person/passenger phone',
       'Demand Source name', 'Driver name', 'Driver external ID',
       'Vehicle plate', 'Price', 'Currency', 'Pooling', 'Rating',
       'Arrived to Pickup (UTC)', 'Pickup Completed (UTC)', 'Pickup address',
       'Pickup coordinates', 'Arrived to Dropoff (UTC)',
       'Dropoff Completed (UTC)', 'Dropoff address', 'Dropoff coordinates',
       'Actual Duration of Ride (min)', 'Planned Distance (m)',
       'Number Of Passengers', 'Number Of Items', 'Created by',
       'Created at (UTC)'],
      dtype='object')
Index(['Ride Type', 'Ride Id', 'Ride External Id', 'Ride Matching Type',
       'Ride Dispatch Type', 'Stop Point Type', 'Stop Point Id',
       'Schedule time (UTC)', 'Schedule arrive after time (UTC)',
       'Schedule arrive before time (UTC)', 'Ride status', 'Stop P

In [104]:
# drop duplitates before join
vehicle_usage_df = vehicle_usage_df.drop_duplicates(subset = ["Ride Id"])
external_df = external_df.drop_duplicates(subset = ["Id"])
vehicle_external_merge = pd.merge(vehicle_usage_df,external_df,how='left',left_on="Ride Id",right_on="Id")
vehicle_external_merge = vehicle_external_merge[vehicle_external_merge["Stop Point Type"] == "dropoff"]
vehicle_external_merge = vehicle_external_merge[["Vehicle Id","Ride Id","Id","External Id"]]
vehicle_external_merge = vehicle_external_merge[~vehicle_external_merge["External Id"].isna()]
vehicle_external_merge = vehicle_external_merge[vehicle_external_merge['External Id'].isin(rides_df['id'])]
vehicle_external_merge

,Vehicle Id,Ride Id,Id,External Id
7,54fc7c8b-940f-4ba6-abda-9237be36e57a,20c4ca15-d7da-4142-b6b3-13413ec263b7,20c4ca15-d7da-4142-b6b3-13413ec263b7,274cdf85-e54d-4ee5-bd36-8a22053382f7
9,54fc7c8b-940f-4ba6-abda-9237be36e57a,9c8638d6-741a-41e2-ada8-5243efe2e256,9c8638d6-741a-41e2-ada8-5243efe2e256,77acd892-49eb-4811-8eb5-0caeb2aadde8
16,54fc7c8b-940f-4ba6-abda-9237be36e57a,f32891d9-611c-4570-a3b7-68ebba859773,f32891d9-611c-4570-a3b7-68ebba859773,e0143e0f-eab6-4f59-b60c-54d1d07a6b84
17,54fc7c8b-940f-4ba6-abda-9237be36e57a,4ba73562-6def-4915-858f-a471044f369b,4ba73562-6def-4915-858f-a471044f369b,cdbc271c-a8cf-45a1-9799-610a7ff7e179
22,54fc7c8b-940f-4ba6-abda-9237be36e57a,8952dd77-0205-46b8-835a-efcf318b7322,8952dd77-0205-46b8-835a-efcf318b7322,2320e312-b980-4be8-aa59-cc8927f88083
...,...,...,...,...
17427,NaN,c6d40805-0fc6-417c-98f0-f19fb6c18757,c6d40805-0fc6-417c-98f0-f19fb6c18757,0dfea269-fc90-485d-9cc9-1938b549f93a
17433,NaN,2da36537-ab73-4081-82bc-8af49ba90ce9,2da36537-ab73-4081-82bc-8af49ba90ce9,024ae5b5-6071-4543-b32e-fdd9d1e219ef
17434,NaN,e5f7bd33-4832-4d3f-986a-3d97f6f40a75,e5f7bd33-4832-4d3f-986a-3d97f6f40a75,d1a065bd-4231-4606-8764-bfdd0a166913
17435,NaN,90a67429-69bb-49f2-b4a9-a416bd98ec6b,90a67429-69bb-49f2-b4a9-a416bd98ec6b,344405f7-8c1f-4d62-ae05-3fcff9b5421d


In [98]:
rides_df = rides_df[~rides_df["id"].isna()]
rides_df = rides_df.drop_duplicates(subset = ["id"])
rides_df.shape

(10744, 51)

In [106]:
rides_vehicle_df = pd.merge(rides_df,vehicle_external_merge,how='left',left_on="id",right_on="External Id", validate="one_to_one")
no_rides_vehicle_df = rides_vehicle_df[rides_vehicle_df["Vehicle Id"].isna()]
no_rides_vehicle_df



,Unnamed: 0,id,user_id,distance,number_of_passenger,price_operations,price_offer,price_payed,free_ride,payment_type,pickup_address,dropoff_address,state,created_from_offer,created_at,scheduled_to,dispatched_at,pickup_arrival_time,arriving_push,vehicle_arrived_at,earliest_pickup_expectation,pickup_first_eta,pickup_eta,pickup_at,dropoff_first_eta,dropoff_eta,dropoff_at,updated_at,arrival_deviation,waiting_time,boarding_time,ride_time,trip_time,shortest_ridetime,delay,longer_route_factor,arrival_indicator,rating,rating_puenktlichkeit,rating_sauberkeit,rating_fahrer,rating_find_modstop,rating_other_comments,cancellation_reason,cancellation_comment,bahn_card_number,year_card_type,year_card_number,canceled_at,rating_question_one,rating_question_two,Vehicle Id,Ride Id,Id,External Id
1,1,18fec0a6-b7ba-442b-8472-04bdb6ba1b86,51e1a1a8-995c-488c-84ce-3789e46f0417,3575,1.0,0.00,2.77,0.00,0.0,BAHN_CARD,Julius-Wilde-Straße,Haltweg,canceled,NaN,2021-06-30 22:51:53,2021-07-01 08:50:00,2021-07-01 08:42:00,NaN,NaN,NaN,2021-07-01 08:45:00,NaN,NaN,NaN,NaN,NaN,NaN,2021-07-01 08:16:03,NaN,NaN,NaN,NaN,NaN,00:07:09,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ed9a1b6c-d543-44c5-954c-f9bba8201f17,ed9a1b6c-d543-44c5-954c-f9bba8201f17,18fec0a6-b7ba-442b-8472-04bdb6ba1b86
3,3,3cffa0f3-e278-4828-b0a1-f55cb35c1adb,44f61d06-8e79-42c6-9abd-0e85fcaf9d6d,7233,1.0,0.00,5.55,0.00,1.0,STANDARD,Weinland Königsbach,Le Quartier Hornbach,canceled,NaN,2021-07-01 11:24:01,NaN,2021-07-01 11:24:01,NaN,NaN,NaN,2021-07-01 11:27:01,2021-07-01 11:37:59,2021-07-01 11:38:00,NaN,2021-07-01 11:50:39,2021-07-01 11:50:40,NaN,2021-11-15 11:24:38,NaN,NaN,NaN,NaN,NaN,00:14:27.960000,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,f797b4d6-2fd4-442a-aec3-32c7f34c9b3e,7b093b19-1bc2-4bc0-b9cb-fcb8cfb8f074,6511,1.0,5.20,3.10,3.10,0.0,VRN,Schütt,Kanzelkopfstraße,completed,NaN,2021-07-01 17:11:11,NaN,2021-07-01 17:11:11,00:10:45,2021-07-01 17:18:56,2021-07-01 17:21:56,2021-07-01 17:14:11,2021-07-01 17:24:39,2021-07-01 17:23:01,2021-07-01 17:22:06,2021-07-01 17:36:25,2021-07-01 17:34:47,2021-07-01 17:32:41,2021-11-15 11:24:38,0.0,00:07:45,00:00:10,00:10:35,00:18:20,00:13:01.320000,00:05:19,0.81,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,9,0f12f88d-00b8-4bb3-a231-683117c0b075,f07028da-ca7e-4713-9e45-743c71712e80,3040,1.0,3.45,1.55,1.55,0.0,VRN,Jüdischer Friedhof,Globus,completed,NaN,2021-07-01 23:33:44,2021-07-02 07:40:00,2021-07-02 07:32:00,00:08:46,2021-07-02 07:37:46,2021-07-02 07:40:46,2021-07-02 07:35:00,2021-07-02 07:40:32,2021-07-02 07:43:36,2021-07-02 07:42:23,2021-07-02 07:48:54,2021-07-02 07:51:58,2021-07-02 07:49:49,2021-11-15 11:24:38,0.0,00:05:46,00:01:37,00:07:26,00:13:12,00:06:04.800000,00:07:07,1.22,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,10,12011bcd-9ee9-4d1a-aba2-b3a5f9097a53,d2a75ace-8b9f-4b32-9914-85e434ce9019,3048,1.0,3.45,3.45,3.45,0.0,STANDARD,Hauptfeuerwache,Grüne Insel,completed,NaN,2021-07-01 23:03:35,2021-07-02 08:20:00,2021-07-02 08:12:00,00:02:55,2021-07-02 08:12:04,2021-07-02 08:14:55,2021-07-02 08:15:00,2021-07-02 08:14:59,2021-07-02 08:15:58,2021-07-02 08:14:55,2021-07-02 08:22:15,2021-07-02 08:23:14,2021-07-02 08:21:06,2021-11-15 11:24:38,-9.0,00:00:00,00:00:00,00:06:11,NaN,00:06:05.760000,NaN,1.01,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10739,18972,37eae9a3-bad9-43e2-8f10-9263d5e9d5e4,f07028da-ca7e-4713-9e45-743c71712e80,3328,1.0,3.87,1.72,1.72,0.0,VRN,Jüdischer Friedhof,Globus,completed,NaN,2022-05-30 19:02:45,2022-05-31 07:30:00,2022-05-31 07:22:00,00:11:35,2022-05-31 07:25:27,2022-05-31 07:33:35,2022-05-31 07:25:00,2022-05-31 07:28:09,2022-05-31 07:38:01,202

In [ ]:
rides_vehicle_df["combined_ride"]  = np.where()